In [26]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'
import json
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

# Load your data
with open('/home/sslunder24/project/chatbot/koreanmulti/data/openai-community/gpt2/train_utters.json', 'r') as f:
    dialogues = json.load(f)

chat_len = []

# Load the pre-trained seq2seq model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = FSMTTokenizer.from_pretrained('facebook/wmt19-en-de')
model = FSMTForConditionalGeneration.from_pretrained('facebook/wmt19-en-de').to(device)



for i, dialogue in enumerate(dialogues):
    chat_len.append(len(dialogue))

list_dialogue = []

for dialogue in dialogues:
    for utter in dialogue:
        list_dialogue.append(utter)

def translate_batch(text_list, batch_size):
    translations = []
    for i in tqdm(range(0, len(text_list), batch_size)):
        batch = text_list[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model.generate(**inputs)
        translations.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
    return translations

# Translate input sequences in batches with progress tracking
batch_size = 32
translated_sequences = translate_batch(list_dialogue, batch_size=batch_size)

translated_dialogue=[]

total_idx = 0

for idx in chat_len:
    translated_dialogue.append(translated_sequences[total_idx:total_idx+idx])
    total_idx = total_idx+idx


# Save the translated dialogues to a JSON file
with open('./data/dbmdz/german-gpt2/train_utters.json', 'w') as f:
    json.dump(translated_dialogue, f, indent=4)

print("Translation complete. Translated dialogues saved to 'train_utters.json'.")


Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 14421/14421 [3:41:31<00:00,  1.09it/s]


Translation complete. Translated dialogues saved to 'train_utters.json'.
